In [2]:

import numpy as np
from numpy.linalg import norm
from scipy.linalg import circulant
from scipy.ndimage import gaussian_filter, convolve1d


def get_gaussian_filter(std, filter_len):
    x = np.zeros(filter_len)
    x[0] = 1
    h = gaussian_filter(x, sigma=std, mode='wrap')
    return convolve1d(h, x, mode='wrap')


def filter2matrix(h):
    h_shift = np.concatenate((h[len(h)//2:], h[:len(h)//2]))
    return circulant(h_shift)


def convolve(x, h):
    return convolve1d(x, h, mode='wrap')

def add_noise(x, snr):
    noise = np.random.randn(x.size)
    return x + 10 ** (-snr / 20) * norm(x) / norm(noise) * noise
    

def compute_snr(x_true, x_noisy):
    return 20 * np.log10(norm(x_true) / norm(x_true - x_noisy))


def soft_thresholding(x, tau):
    return np.sign(x) * np.maximum(np.abs(x) - tau, 0)


def ista_l1(f, grad_f, x0, step_size, alpha, n_iterations, return_iterates):
    x = x0.copy()
    residuals = []
    for _ in range(n_iterations):
        x = soft_thresholding(x - step_size * grad_f(x), alpha * step_size)
        residuals.append(f(x))
    if return_iterates:
        return x, residuals
    else:
        return x




print('Testing the ISTA algorithm on a simple example...')
np.random.seed(0)
n = 100
m = 256
k = 10
A = np.random.randn(m, n)
x_true = np.zeros(n)

selected = np.random.choice(n, k, replace=False)


x_true[selected] = np.random.randn(k)
sigma = 1e-1
b = A.dot(x_true) + sigma * np.random.randn(m)
L = norm(A, ord=2) ** 2
x0 = np.zeros(n)
step_size = 1 / L
alpha = 1

f = lambda x: 0.5 * norm(A.dot(x) - b) ** 2
grad_f = lambda x: A.T.dot(A.dot(x) - b)

x_hat, residuals = ista_l1(f, grad_f, x0, step_size, alpha, 1000, True)
print('Error: {}'.format(norm(x_hat - x_true) / norm(x_true)))
print('Residual: {}'.format(residuals[-1]))
print('Done.')





Testing the ISTA algorithm on a simple example...
Error: 0.01934425384137391
Residual: 1.1074689472201342
Done.
